In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import MultiPolygon
from tqdm import tqdm
import gdown
import matplotlib.pyplot as plt
from plotly.offline import init_notebook_mode
init_notebook_mode(connected= True)
import plotly.express as px

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
file_id = "1U34r_SnEZNpIPXM-_lRDjlin3vbTN_sL"
url = f"https://drive.google.com/uc?export=download&id={file_id}"

# Download the file
gdown.download(url, "FD_SALAAN_2021.csv", quiet=False)

In [ ]:
base = pd.read_csv("FD_SALAAN_2021.csv", delimiter=";")

In [ ]:
base.head()

PICO stands for
 • Population : Who are we interested in? : all French population in 2021 (all employed French population ?)
 • Intervention : What treatment/intervention do we study? : do we find a specific treatment ? related to sex/geography/PCS? determined by lasso selection ?
 • Comparison : What are we comparing it to? depends on the treatment
 •Outcome : What are we interested in? wages 

## Exploratory data analysis: descriptive analysis

In [ ]:
base.columns

In [ ]:
base.describe()

In [ ]:
numerical_columns = [
    'DATDEB', 'DATFIN', #date début et fin de rémunération par rapport au 01/01
     'AGE', #age en années
     'POND' #pondération 1/12e
]
categorical_columns = [
    'A6', 'A17', 'A38' #activité en nomenclature agrégrée
    'CPFD', #temps complet ou partiel
    'DEPR', 'DEPT', #département résidence et travail
    'DOMEMPL', 'DOMEMPL_EM', #domaine de l'emploi et l'établissement d'affectation/employeur
     'DUREE', #durée de paie en jours
    'FILT', #indic poste annexe 2 ou non-annexe 1 (seuils rémunération volume)
    'REGR', 'REGT', #région de résidence et de travail
    'SEXE', #1 homme 2 femme
    'PCS', #PCS-ESE
    'TYP_EMPLOI', #ordinaire, apprenti, autre
    'CONV_COLL', #convention collective
    'NBHEUR', 'NBHEUR_TOT', #nombre d'heures salariées total (quelle diff?)
    'TRNNETO', #rémunération nette globale en tranches -> à passer en numérique ?
    'TRALCHT', #total des indémnités de chômage, en tranches -> passage en numérique ?
    'TREFF', #tranche d'effectif : de 0 à 250+ postes
    'CONT_TRAV', #contrat de travail : APP apprentissage, TOA occasionnel ou à l'acte, TTP intérim, AUTre
    'CS', #CSP mais code plus simple 
    'AGE_TR', #age en tranches quadriennales
    'DATDEB_TR',
       'DATFIN_TR', #dates début et fin rémunération en tranches
    'DUREE_TR', #durée de paie exprimée en jours en tranches mensuelles
]

In [ ]:
base['PCS'].describe()

In [ ]:
base['PCS'].value_counts()

In [ ]:
fig, ax = plt.subplots()
ax.hist(base['DATDEB'], bins=100, cumulative=False)
ax.set_xlabel('Date of start of revenue with respect to the 01/01')
ax.set_ylabel('Frequency')
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.hist(base['DATDEB'], bins=100, cumulative=False, log=True)
ax.set_xlabel('Date of start of revenue')
ax.set_ylabel('Frequency')
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.hist(base['DATFIN'], bins=100, cumulative=False)
ax.set_xlabel('Date of end of revenue with respect to the 01/01')
ax.set_ylabel('Frequency')
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.hist(base['DATFIN'], bins=100, cumulative=False, log=True)
ax.set_xlabel('Date of end of revenue with respect to the 01/01')
ax.set_ylabel('Frequency')
plt.show()

In [ ]:
base['DOMEMPL'].value_counts()